In [1]:
!pip install elasticsearch

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 387.9/387.9 kB 4.0 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 4.0 MB/s eta 0:00:00


In [21]:
from elasticsearch import Elasticsearch
import json
import os
import pandas as pd

# Create the client instance
client = Elasticsearch("http://localhost:9200")

# Successful response!
client.info()

/var/folders/nd/mfwt7yf50wdb5y15q3f9mzb80000gn/T/ipykernel_85139/2156522912.py:10: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  client.info()


ObjectApiResponse({'name': '03d673b8d0e2', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'Gz8IOq2-SbyAPmMlvOylhw', 'version': {'number': '7.17.9', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': 'ef48222227ee6b9e70e502f0f0daa52435ee634d', 'build_date': '2023-01-31T05:34:43.305517834Z', 'build_snapshot': False, 'lucene_version': '8.11.1', 'minimum_wire_compatibility_version': '6.8.0', 'minimum_index_compatibility_version': '6.0.0-beta1'}, 'tagline': 'You Know, for Search'})

In [22]:
import os
import json

# Get the documents from the directory
entries = os.listdir('../1-publications/json')
for i in entries:
    # Opening JSON file
    with open('../1-publications/json/' + i) as f:
        try:
            # returns JSON object as a dictionary
            data = json.load(f)

            # Remove the unnecessary fields
            fields_to_remove = ['url', 'paperId', 'year', 'publicationDate', 'authors']
            for field in fields_to_remove:
                data.pop(field, None)

            # Index the document
            client.index(index='prueba', document=data)
        except json.JSONDecodeError:
            print(f"Error decoding JSON file: {i}")

# Refresh the index
client.indices.refresh(index='prueba')


Error decoding JSON file: 80313ba1f12e4525b941ba29f8e020cf5ae8b835.json


/var/folders/nd/mfwt7yf50wdb5y15q3f9mzb80000gn/T/ipykernel_85139/1534271428.py:19: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  client.index(index='prueba', document=data)
/var/folders/nd/mfwt7yf50wdb5y15q3f9mzb80000gn/T/ipykernel_85139/1534271428.py:24: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  client.indices.refresh(index='prueba')


ObjectApiResponse({'_shards': {'total': 2, 'successful': 1, 'failed': 0}})

In [23]:
# Query for the keyword you want
val = input("Enter the keyword you are looking for: ")
result = client.search(
 index='prueba',
  query={
    'match': {'abstract': val}
  }
 )
a = result['hits']['hits']

data = []

for hit in a:
    entry = {
        'title': hit['_source']['title'],
        'abstract': hit['_source']['abstract'],
        'score': hit['_score']
    }
    data.append(entry)

df = pd.DataFrame(data, columns=['title', 'abstract', 'score'])
df

/var/folders/nd/mfwt7yf50wdb5y15q3f9mzb80000gn/T/ipykernel_85139/3414197585.py:3: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  result = client.search(


,title,abstract,score
0,Overview of Big Data and Its Visualization,Big Data is data sets that are so voluminous a...,0.205878
1,"Comparison between Expert Systems, Machine Lea...","Today, the science of artificial intelligence ...",0.203744
2,Skin Cancer Diagnosis Using an Improved Ensemb...,In recent years skin cancer is becoming more a...,0.200586
3,"Soluble ST2, a biomarker of fibrosis, is assoc...",Abstract Several diseases have a deleterious f...,0.187385


In [20]:
# DO NOT RUN THIS CODE IF YOU DON'T WANT TO DELETE THE INDEX!!!
client.options(ignore_status=[400,404]).indices.delete(index='prueba')

/var/folders/nd/mfwt7yf50wdb5y15q3f9mzb80000gn/T/ipykernel_85139/986347348.py:2: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  client.options(ignore_status=[400,404]).indices.delete(index='prueba')


ObjectApiResponse({'acknowledged': True})